In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import wandb
import torch
import torch.nn as nn
from torch.nn import functional as F

import pytorch_lightning as pl

import utils
import ud
import neural

C:\Users\elaza\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:23: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


In [3]:
class ConlluDataModule(pl.LightningDataModule):

    def __init__(self, train_filename, test_filename, batch_size=64, validation_size=300):
        # TODO: use dev set instead of splitting validation
        super().__init__()
        SENTENCE_MAXLEN = 30
        WORD_MAXLEN = 11
        self.validation_size = validation_size
        self.batch_size = batch_size
        
        train_data_x, train_data_y = ud.load_dataset(train_filename, ud.parse_opnlp, SENTENCE_MAXLEN, WORD_MAXLEN)
        self.data_train_full = torch.utils.data.TensorDataset(torch.Tensor(train_data_x), *[torch.Tensor(y).to(torch.int64) for y in train_data_y])
        
        test_data_x, test_data_y = ud.load_dataset(test_filename, ud.parse_opnlp, SENTENCE_MAXLEN, WORD_MAXLEN)
        self.data_test = torch.utils.data.TensorDataset(torch.Tensor(test_data_x), *[torch.Tensor(y).to(torch.int64) for y in test_data_y])

    def prepare_data(self):
        # No state assignment here
        pass

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            self.data_train, self.data_val = torch.utils.data.random_split(self.data_train_full, [self.validation_size, len(self.data_train_full) - self.validation_size])
            self.dims = tuple(self.data_train[0][0].shape)

        if stage == 'test' or stage is None:
            self.dims = tuple(self.data_test[0][0].shape)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.data_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.data_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.data_test, batch_size=self.batch_size)

In [4]:
NUM_EMBEDDING = 2000


class SentenceModel(neural.UdModel):

    def __init__(self, label_map, units=400, learning_rate=2e-3):
        super().__init__()
        self.save_hyperparameters()
        
        self.units = units

        self.embed = nn.Embedding(num_embeddings=NUM_EMBEDDING, embedding_dim=units)
        
        self.char_lstm = neural.SumSharedBiLSTM(units)
        
        self.word_lstm = neural.SumBiLSTM(units)
        
        self.tasks = nn.ModuleDict([(label_name, nn.Linear(in_features=units, out_features=class_size))
                                     for label_name, class_size in label_map.items()])
        
        self.norm = nn.BatchNorm1d(units, affine=False)

    def forward(self, x):
        BATCH_SIZE, SENT_MAXLEN, WORD_MAXLEN = x.shape
        
        x = x.to(torch.int64)
        # x: (BATCH_SIZE, SENT_MAXLEN, WORD_MAXLEN)
        
        # Step 0: character embedding 
        
        x = x.reshape(BATCH_SIZE * SENT_MAXLEN, WORD_MAXLEN)
        # x: (BATCH_SIZE * SENT_MAXLEN, WORD_MAXLEN)

        embeds = self.embed(x)
        # embeds: (BATCH_SIZE * SENT_MAXLEN, WORD_MAXLEN, UNITS)
        
        embeds = embeds.permute([1, 0, 2])
        # x: (WORD_MAXLEN, BATCH_SIZE * SENT_MAXLEN, UNITS)

        
        # STEP 1: character-level lstm -> word embedding
        
        _, char_hidden = self.char_lstm(embeds)
        # char_hidden: (BATCH_SIZE * SENT_MAXLEN, UNITS)
        char_hidden = self.norm(char_hidden)
        
        char_hidden = char_hidden.reshape(BATCH_SIZE, SENT_MAXLEN, -1)
        # char_hidden: (BATCH_SIZE, SENT_MAXLEN, UNITS)
        
        char_hidden = char_hidden.permute([1, 0, 2])
        # char_hidden: (SENT_MAXLEN, BATCH_SIZE, UNITS)

        # STEP 2: sequence tagging using word-level lstm
    
        word_lstm_out, _ = self.word_lstm(char_hidden)
        # word_lstm_out: (SENT_MAXLEN, BATCH_SIZE, UNITS)
        
        word_lstm_out = word_lstm_out.permute([1, 0, 2])
        # word_lstm_out: (BATCH_SIZE, SENT_MAXLEN, UNITS)
        
        char_hidden = char_hidden.permute([1, 0, 2])
        # char_hidden: (BATCH_SIZE, SENT_MAXLEN, UNITS)
        
        summed = word_lstm_out + char_hidden
        # summed: (BATCH_SIZE, SENT_MAXLEN, UNITS)
        
        return {name: linear(summed).permute([0, 2, 1])
                for name, linear in self.tasks.items()}
  
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)
        # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [4], gamma=0.5)
        return [optimizer] # , [scheduler]


In [5]:
dataset = ConlluDataModule(
    f'../Hebrew_UD/he_htb-ud-train.conllu',
    f'../Hebrew_UD/he_htb-ud-dev.conllu',
    batch_size=64)
dataset.setup()

In [7]:
model = SentenceModel(ud.Token.label_map(), units=400, learning_rate=3e-3)
model.char_lstm.load_state_dict(torch.load('models/verbs_sumsharedlstm.pth'))
wandb_logger = pl.loggers.WandbLogger(project='rootem', group='ud-fix1', name=f'full')
trainer = pl.Trainer(gpus=1, max_epochs=50, logger=wandb_logger)
trainer.fit(model, dataset)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]



  | Name      | Type            | Params
----------------------------------------------
0 | embed     | Embedding       | 800 K 
1 | char_lstm | SumSharedBiLSTM | 2 M   
2 | word_lstm | SumBiLSTM       | 2 M   
3 | tasks     | ModuleDict      | 88 K  
4 | norm      | BatchNorm1d     | 0     


C:\Users\elaza\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:23: RuntimeWarning: The metric you returned None must be a `torch.Tensor` instance, checkpoint not saved HINT: what is the value of loss in validation_epoch_end()?
  warnings.warn(*args, **kwargs)
C:\Users\elaza\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\utilities\distributed.py:23: RuntimeWarning: Can save best model only with loss available, skipping.
  warnings.warn(*args, **kwargs)


1

In [98]:
sent = 'החשודים המשיכו בנסיעה ואחרי 700 מטר עצרו ליד בית נוסף, וירו עשרות קליעים על מכונית וקירות הבית. בני המשפחה הספיקו להימלט מהמקום.'.split()
text = torch.Tensor([[utils.encode_word(word, 11) for word in sent]] * 2)  # .cuda()

In [99]:
res = [[ud.Token.decode_label(label, idx) for idx in value.argmax(1)[0]]
        for label, value in model(text).items()]
print('Token', *[f[:7] for f in ud.Token.labels()], sep='\t')
for item in zip(sent, *res):
    print(*item, sep='\t')


Token	Abbr	Case	Cconj	Det	Definit	Gender	HebExis	HebSour	HebBiny	Mood	Number	Person	Polarit	Pos	Prefix	PronGen	PronNum	PronPer	PronTyp	Reflex	R1	R2	R3	R4	Tense	VerbFor	VerbTyp	Voice	Xtra
החשודים	No	Gen	False	True	Cons	Masc	No	ConvUncertainLabel	PAAL	No	Plur	1,2,3	Pos	NOUN	No	Masc	Plur	3	Int	No	ח	שׂ	ר	ד	Past	Part	Cop	Act	No
המשיכו	No	Acc	False	True	Def	Fem,Masc	No	ConvUncertainLabel	HIFIL	No	Plur	3	Pos	VERB	No	Masc	Sing	3	Int	No	מ	ש	.	כ	Past	Part	Mod	Act	Junk
בנסיעה	No	Acc	False	True	Def	Fem	No	ConvUncertainHead	NIFAL	No	Sing	3	Neg	NOUN	No	Fem	Sing	3	Prs	No	ב	ס	ס	ע	Past	Inf	Cop	Mid	No
ואחרי	No	Acc	True	False	Cons	Masc	No	ConvUncertainLabel	PAAL	Imp	Plur	1	Neg	SCONJ	Yes	Fem,Masc	Sing	1	Dem	No	א	ז	ר	א	_	Part	Mod	Act	No
700	Yes	Tem	False	False	_	_	True	_	PAAL	_	Plur,Sing	_	_	NUM	Yes	Fem,Masc	Plur,Sing	2	Dem	_	ג	י	ד	צ'	Present	_	_	_	Junk
מטר	Yes	Gen	False	False	Cons	Masc	No	ConvUncertainHead	PAAL	No	Sing	3	Neg	DET	Yes	Fem	Sing	2	Int	No	ט	ט	פ	ר	Past	Part	Mod	Act	No
עצרו	No	Acc	False	False	De

In [ ]:
for s, p, b, *rs in zip(sent, pos, binyan, r1, r2, r4):
    print(s, p, b, ''.join(rs), sep='\t')

In [ ]:
model = IndependentModel(300)
trainer = pl.Trainer(gpus=1, auto_lr_find=True, max_epochs=10)
lr_finder = trainer.lr_find(model, train_dataloader=dataset, min_lr=1e-5, early_stop_threshold=None)

In [ ]:
fig = lr_finder.plot(suggest=True)

In [ ]:
trainer.test(model, test_dataloaders=dataset.test_dataloader())

In [77]:
dataset.data_train[:][2]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [ ]:
label_map = {name: ud.Token.class_size(name) for name in [
#     'Abbr',
    'Case',
    'Cconj',
    'Det',
#     'Definite',
    'Gender',
#     'HebExistential',
#     'HebSource',
    'HebBinyan',
#     'Mood',
    'Number',
    'Person',
#     'Polarity',
    'Pos',
#     'Prefix',
    'PronGender',
    'PronNumber',
    'PronPerson',
    'PronType',
#     'Reflex',
    'R1',
    'R2',
    'R3',
    'R4',
    'Tense',
#     'VerbForm',
    'VerbType',
    'Voice',
]}